# Hyperparameter Optimization for SARIMA Time-Series Forecasting
Tuomas Eerola - 2019

## Download Techila SDK into your Google Colab.

In [0]:
my_server_ip = '' #@param {type:"string"}
my_password = ''  #@param {type: "string"}
!wget --post-data="login=admin&password={my_password}" --no-check-certificate https://{my_server_ip}/gce_downloadsdk.php -O techilasdk.zip
!unzip techilasdk.zip

%cd /content/techila/lib/python3
!python setup.py install
!echo "JVMPATH=/usr/lib/jvm/default-java/lib/server/libjvm.so" > /content/techila/lib/TechilaDLL.conf

## Launch some Techila Workers
Run and click the link to launch some Techila Workers. Techila Workers are the compute nodes in your Techila Distributed Computing Engine system that do the actual processing.

In [0]:
print('https://' + my_server_ip + '/gce_deployment.php?step=deploy')

## Load IoT sensor data and prepare it.

In [0]:
import pandas as pd

from urllib.request import urlopen

log_url = "http://eerola.dy.fi/temp/temperature.log"

series_own = pd.read_csv(log_url, sep=" ", parse_dates=[[0, 1]])
series_own.columns=['Date Time', 'SourceInfo1', 'SourceInfo2', 'MeasurementInfo1', 'Temp', 'MeasurementInfo2', 'Measurement2']
dropcolumns = ['SourceInfo1', 'SourceInfo2', 'MeasurementInfo1', 'MeasurementInfo2', 'Measurement2']
series_own.drop(dropcolumns, inplace=True, axis=1)
series_own.set_index('Date Time', inplace=True)

series = series_own.reset_index()
series = series.drop_duplicates(subset='Date Time', keep='last')
series = series.set_index('Date Time')
series = series.resample('H').bfill()

print ("Data loading ready.")

## Configure the run settings.

Select how much of the IoT sensor data we want to use in the hyperparameter optimization, and other configuration items.

In [0]:
total_number_of_data_points = 100
test_data_percentage_of_data_points = 20

# When setting run_parallel False, comment out
# @techila.distributable() line in the helper functions.
run_parallel = True

# Grouping more operations in one parallel job can benefit the performance.
num_of_operations_per_parallel_job = 20

print ("Ready.")

## Introduce the set of SARIMA configs to try.

Configuring a SARIMA requires selecting hyperparameters for both the trend and seasonal elements of the series.

There are three trend elements that require configuration.

They are the same as the ARIMA model; specifically:

*   p: Trend autoregression order.
*   d: Trend difference order.
*   q: Trend moving average order.


There are four seasonal elements that are not part of ARIMA that must be configured; they are:

*   P: Seasonal autoregressive order.
*   D: Seasonal difference order.
*   Q: Seasonal moving average order.
*   m: The number of time steps for a single seasonal period.

In [0]:
def sarima_configs(seasonal=[0]):
  models = list()

  p_params = [0, 1, 2]
  d_params = [0, 1]
  q_params = [0, 1, 2]
  t_params = ['n','c','t','ct']
  P_params = [0, 1, 2]
  D_params = [0, 1]
  Q_params = [0, 1, 2]
  m_params = seasonal

  for p in p_params:
    for d in d_params:
      for q in q_params:
        for t in t_params:
          for P in P_params:
            for D in D_params:
              for Q in Q_params:
                for m in m_params:
                  cfg = [(p,d,q), (P,D,Q,m), t]
                  models.append(cfg)
  return models

print ("Ready.")

## Introduce some helper functions.

Run this to introduce some helper functions that we will use in this code.

In [0]:
if total_number_of_data_points > len(series):
  print ("Error. Reduce total_number_of_data_points.")
elif test_data_percentage_of_data_points >100:
  print ("Error. Reduce test_data_percentage_of_data_points.")
else:
  data = series['Temp'].tail(total_number_of_data_points).tolist()
  n_test_percentage = test_data_percentage_of_data_points/100
  
from statsmodels.tsa.statespace.sarimax import SARIMAX
from math import sqrt
from sklearn.metrics import mean_squared_error
from warnings import catch_warnings
from warnings import filterwarnings
import techila

# one-step sarima forecast
def sarima_forecast(history, config):
  order, sorder, trend = config
	# define model
  model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
  model_fit = model.fit(disp=False)
	# make one step forecast
  yhat = model_fit.predict(len(history), len(history))
  return yhat[0]

# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
  predictions = list()
	# split dataset
  train, test = train_test_split(data, n_test)
	# seed history with training dataset
  history = [x for x in train]
	# step over each time-step in the test set
  for i in range(len(test)):
		# fit model and make forecast for history
    yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
    predictions.append(yhat)
		# add actual observation to history for the next loop
    history.append(test[i])
	# estimate prediction error
  error = measure_rmse(test, predictions)
  return error

# score a model, return None on failure
@techila.distributable()
def score_model(data, n_test, cfg, debug=False):
  result = None
	# convert config to a key
  key = str(cfg)

	# show all warnings and fail on exception if debugging
  if debug:
    result = walk_forward_validation(data, n_test, cfg)
  else:
		# one failure during model validation suggests an unstable config
    try:
			# never show warnings when grid searching, too noisy
      with catch_warnings():
        filterwarnings("ignore")
        result = walk_forward_validation(data, n_test, cfg)
    except:
      error = None
	# check for an interesting result
  #if result is not None:
  #  print(' > Model[%s] %.3f' % (key, result))
  return (key, result)

# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
  scores = None
  result = []
  for cfg in cfg_list:
    result.append(score_model(data, n_test, cfg))

  if parallel:
    techila.run(steps=num_of_operations_per_parallel_job)
    scores = [x.result for x in result]
  else:
    scores = [x for x in result]
    
  # remove empty results
  scores = [r for r in scores if r[1] != None]
  # sort configs by error, asc
  scores.sort(key=lambda tup: tup[1])
  return scores

print ("Ready.")

## Search the Top #3 Hyperparameter candidates
Use these parameters to run your forecasts.

In [0]:
# Do data splitting
n_test = round(len(data) * n_test_percentage)
# Load model configudations
cfg_list = sarima_configs()

# Run grid search
scores = grid_search(data, cfg_list, n_test, run_parallel)

# Show the top 3 hyperparameter configudations
print("Top #3 Hyperparameter candidates:")
for cfg, error in scores[:3]:
  print(cfg, error)